# Prototyping with Python

In this book, we use Python to implement automated testing techniques, and also as the language for most of our test subjects.  Why Python?  The short answer is

> Python made us amazingly _productive_.  Most techniques in this book took **2-3 days** to implement.  This is about **10-20 times faster** than for "classic" languages like C or Java.

A factor of 10–20 in productivity is enormous, almost ridiculous.  Why is that so, and which consequences does this have for research and teaching?

## Python is Easy

Python is a high-level language that allows one to focus on the actual _algorithms_ rather than how individual bits and bytes are passed around in memory.  For this book, this is important: We want to focus on how individual techniques work, and not so much their optimization.  Focusing on algorithms allows you to toy and tinker with them, and quickly develop your own.  Once you have found out how to do things, you can still port your approach to some other language or specialized setting.

As an example, take the (in)famous triangle program, which classifies a triangle of lengths $a$, $b$, $c$ into one of three categories.  It reads like pseudocode; yet, we can easily execute it.

In [ ]:
def triangle(a, b, c):
    if a == b:
        if b == c:
            return 'equilateral'
        else:
            return 'isosceles'
    else:
        if b == c:
            return 'isosceles'
        else:
            if a == c:
                return 'isosceles'
            else:
                return 'scalene'

Here's an example of executing the `triangle()` function:

In [ ]:
triangle(2, 3, 4)

For the remainder of this chapter, we will use the `triangle()` function as ongoing example for a program to be tested.  Of course, the complexity of `triangle()` is a far cry from large systems, and what we show in this chapter will not apply to, say, an ecosystem of thousands of intertwined microservices.  Its point, however, is to show how easy certain techniques can be – if you have the right language and environment.

## Dynamic Analysis in Python is Extremely Easy

Dynamic analysis is the ability to track what is happening during program execution.  The Python `settrace()` mechanism allows you to track all code lines, all variables, all values, as the program executes – and all this in a handful of lines of code.  Our `Coverage` class from [the chapter on coverage](Coverage.ipynb) shows how to capture a trace of all lines executed in five lines of code; such a trace easily converts into sets of lines or branches executed.  With two more lines, you can easily track all functions, arguments, variable values, too – see for instance our [chapter on dynamic invariants](DynamicInvariants).  And you can even access the source code of individual functions (and print it out, too!)  All this takes 10, maybe 20 minutes to implement.

Here is a piece of Python that does it all.  We track lines executed, and for every line, we print its source codes and the current values of all local variables:

In [ ]:
import sys
import inspect

In [ ]:
def traceit(frame, event, arg):
    function_name = frame.f_code.co_name
    lineno = frame.f_lineno
    
    if function_name == 'triangle':
        source = inspect.getsource(globals()[function_name]).splitlines()
        print("%s:%d:%s" % (function_name, lineno, source[lineno]))
        for var in frame.f_locals:
            print(var, "=", frame.f_locals[var])
    return traceit

The function `sys.settrace()` registers `traceit()` as a trace function; it will then trace the given invocation of `triangle()`:

In [ ]:
sys.settrace(traceit)
triangle(2, 2, 1)
sys.settrace(None)

In comparison, try to build such a dynamic analysis for, say, C.  You can either _instrument_ the code to track all lines executed and record variable values, storing the resulting info in some database.  This will take you _weeks,_ if not _months_ to implement.  You can also run your code through a debugger (step-print-step-print-step-print); but again, programming the interaction can take days.  And once you have the first results, you'll probably realize you need something else or better, so you go back to the drawing board.  Not fun.

## Static Analysis in Python can be Easy

Static analysis refers to the ability to analyze _program code_ without actually executing it.

If your static analysis does not have to be sound, though – for instance, because you only use it to _support_ and _guide_ another technique such as  testing – then a static analysis in Python can be very simple.  The `ast` module allows you to turn any Python function into an abstract syntax tree (AST), which you then can traverse as you like.  Here's the AST for our `triangle()` function:

In [ ]:
import ast
import astor
import showast

In [ ]:
triangle_source = inspect.getsource(triangle)
triangle_ast = ast.parse(triangle_source)
showast.show_ast(triangle_ast)

Now suppose one wants to identify all `triangle` branches and their conditions using static analysis.  You would traverse the AST, searching for `If` nodes, and take their first child (the condition).  This is easy as well:

In [ ]:
class ConditionVisitor(ast.NodeVisitor):
    def __init__(self):
        self.conditions = []

    def visit_If(self, node):
        self.conditions.append(astor.to_source(node.test).strip())
        self.generic_visit(node)

Here are the four `if` conditions occurring in the `triangle()` code:

In [ ]:
visitor = ConditionVisitor()
visitor.visit(triangle_ast)
visitor.conditions

Not only can we extract individual program elements, we can also change them at will and convert the tree back into source code.  Program transformations (say, for instrumentation or mutation analysis) are a breeze.  The above code took five minutes to write.  Again, try that in Java or C.

## There's a Python interface for Everything

Let's get back to testing.  We have shown how to extract conditions from code.  To reach a particular location in the `triangle()` function, one needs to find a solution for the _path conditions_ leading to that branch.  To reach the last line in `triangle()` (the `'scalene'` branch), we have to find a solution for $$a \ne b \land b \ne c \land a \ne c$$.  We can make use of a _constraint_ solver for this, such as Microsoft's [_Z3_ solver](https://github.com/Z3Prover/z3):

In [ ]:
import z3

Let us use Z3 to find a solution for the `'scalene'` branch condition:

In [ ]:
a = z3.Int('a')
b = z3.Int('b')
c = z3.Int('c')

In [ ]:
z3.solve(z3.And(a > 0, b > 0, c > 0, a != b, b != c, a != c))

We can use this solution right away for testing the `triangle()` function and find that it indeed covers the `'scalene'` branch.

In [ ]:
triangle(1, 2, 3)

Now, all it takes to produce a symbolic tester is to find the possible paths through the syntax tree and solve the conjunction of the path conditions, converted to Z3 format.

In [ ]:
visitor.conditions

In [ ]:
def negate(conditions, conditions_to_negate):
    new_conditions = []
    for i, cond in enumerate(conditions):
        if i in conditions_to_negate:
            cond = 'z3.Not(' + conditions[i] + ')'
        new_conditions.append(cond)
    return new_conditions

In [ ]:
negate(visitor.conditions, {0})

In [ ]:
negate(visitor.conditions, {1, 2})

In [ ]:
def z3cond(conditions, conditions_to_negate):
    return ", ".join(negate(conditions, conditions_to_negate))

In [ ]:
z3cond(visitor.conditions, {0, 1, 2})

In [ ]:
eval('z3.solve(z3.And(a > 0, b > 0, c > 0, ' + z3cond(visitor.conditions, {0, 1, 2}) + '))')

In [ ]:
def triangle_test():
    ...
    # How about traversing the branches of the pgm?

To make it easier for a constraint solver to find solutions, you could also provide _concrete values_ observed from earlier executions that already are known to reach specific paths in the program.  Such concrete values would be gathered from the tracing mechanisms above, and boom: you would have a pretty powerful and scalable concolic (concrete-symbolic) test generator.

Now, the above might take you a day or two, and as you expand your test generator beyond `triangle()`, you will add more and more features.  The nice part is that every of these features you will invent might actually be a research contribution – something nobody has thought of before.  Whatever idea you might have: you can quickly implement it and try it out in a prototype.  And again, this will be orders of magnitude faster than for conventional languages.

## Things that will not work

Python has a reputation for being hard to analyze statically, and this is true; its dynamic nature makes it hard for traditional static analysis to exclude specific behaviors.  

We see Python as a great language for prototyping automated testing and dynamic analysis techniques, and as a good language to illustrate _lightweight_ static and symbolic analysis techniques that would be used as _guidance_ and _support_ for other techniques (say, for generating software tests).  But if you want to _prove_ specific properties (or the absence thereof) by static analysis of code only, Python is a challenge, to say the least; and there are areas for which we would definitely _warn_ against using it.  

### (No) Type Checking

Using Python to demonstrate _static type checking_ will be suboptimal (to say the least) because, well, Python programs typically do not come with type annotations.  You _can_, of course, annotate variables with types, as we assume in the [chapter on Symbolic Fuzzing](SymbolicFuzzer.ipynb):

In [ ]:
def typed_triangle(a: int, b: int, c: int) -> str:
    return triangle(a, b, c)

Most real-world Python code will not be annotated with types, though.  While you can also _retrofit them_, as discussed in [our chapter on dynamic invariants](DynamicInvariants.ipynb), Python simply is not a good domain to illustrate type checking. If you want to show the beauty and usefulness of type checking, use a strongly typed language like Java or Haskell.

### (No) Program Proofs

Python is a highly dynamic language in which you can change _anything_ at runtime.  It is no problem assigning a variable different types, as in

In [ ]:
x = 42
x = "a string"

or change the existence (and scope) of a variable depending on some runtime condition:

In [ ]:
p1, p2 = True, False

if p1:
    x = 42
if p2:
    del x
    
# Does x exist at this point?

Such properties make symbolic reasoning on code (including static analysis and type checking) much harder, if not outright impossible.  If you need lightweight static and symbolic analysis techniques as _guidance_ for other techniques (say, as test generation), then imprecision may not hurt much; but if you want to derive _guarantees_ from your code, do not use Python as test subject; again, strongly statically typed languages like Java and Haskell are much better grounds for experimentation.

This does not mean that languages like Python should _not_ be statically checked.  On the contrary, the widespread usage of Python calls loudly for better static checking tools.  But if you want to teach or research static and symbolic techniques, we definitely would not use Python as our language of choice.